In [2]:
import psycopg2
import pandas as pd
import time
from config import host, user, user1, password, password1

In [3]:
start_time = time.time()

In [4]:
#PostgreSQL Connection
try:
    pg_db = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db.autocommit = True
    pgcursor = pg_db.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

#PostgreSQL Connection 2
try:
    pg_db1 = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db1.autocommit = True
    pgcursor1 = pg_db1.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

PostgreSQL Connection Established
PostgreSQL Connection Established


In [ ]:
count_success = 0
Total_Records_from_source = 0
count_failed_emails = 0

#Selects data from table
slt_cmd ="Select Distribution_OrderInfo_ID,ISBN,ARCHIVED,EBOOK,LIST_PRICE,ORDER_STATUS,ORDER_STATUS_DATE, \
TITLE,DISTRIBUTION_ORDER_INSTORES_ID,DISTRIBUTION_METADATA_ID,ID,ORDER_DATE,CCID,FORMAT, \
UPDATED_STATUS,CREATED_DATE,UPDATED_DATE from stg_distribution_orderinfo"
pgcursor.execute(slt_cmd)

#Extracting column names 
col_names_lst = [i[0] for i in pgcursor.description] 
col_names = ', '.join(col_names_lst)

#Inserting data from PostgreSQL STG into PostgreSQL DW
placeholders = ', '.join(['%s'] * (len(col_names_lst)) ) 

#Inserting data into PostgreSQL DW
insrt_cmd = "INSERT INTO DW_Distribution_Order_Info_DIM(Distribution_OrderInfo_ID,ISBN,ARCHIVED,EBOOK,LIST_PRICE,ORDER_STATUS,ORDER_STATUS_DATE, \
TITLE,DISTRIBUTION_ORDER_INSTORES_ID,DISTRIBUTION_METADATA_ID,CUSTOMER_ID,ORDER_DATE,CCID,FORMAT, \
UPDATED_STATUS,CREATED_DATE,UPDATED_DATE) " \
            "VALUES ( %s )" % (placeholders)
# print(insrt_cmd)

for row in pgcursor:
    try:
        Total_Records_from_source = pgcursor.rowcount
        pgcursor1.execute(insrt_cmd,row)
        count_success += 1
    
    except Exception as e:
        count_failed_emails += 1 
        
insrt_log = "INSERT INTO LOG_DIM (Phase, Source_Table_Name, Target_Table_Name, Total_Records_Source_Table, \
Total_Records_Target_Table,New_Records_Source,New_Records_Target, Status, Remarks, Execution_time) \
Values ('StoDW','stg_DISTRIBUTION_ORDERINFO','DW_Distribution_Order_Info_DIM'," +str(Total_Records_from_source)\
+", "+str(count_success)+",'Completed','" +str(count_failed_emails)+" Failed','"+str(round(time.time() - start_time,2))+" seconds')"
pgcursor.execute(insrt_log)

pg_db.close()
pg_db1.close()